# User management basic
Using the API documented at: <a href="https://cloud.ibm.com/apidocs/cloud-pak-data/cloud-pak-data-4.5.0" target="_blank">IBM Cloud Pak for Data Platform API</a><br/>
This is the latest despite CPD 4.6 being out.

Capabilities demonstrated:
- Defining python support functions
- Getting an access token
- List all users
- Create a user
- Delete the newly created user

In [ ]:
import json
import sys
import requests
from datetime import datetime

### Make sure to set the variables in the next cell
cpd_url, username, password

In [ ]:
# cluster URL, make sure it ends with "/", and no "zen" ending
# example:
#cpd_url = "https://cpd-cpd.ai-governance-12345a678e90addd123c4567c8f9a012-3456.us-east.containers.appdomain.cloud/"
cpd_url = "https://cloud-pak-for-data/"

username = "<username>"
password = "<password>"

## Support functions

In [ ]:
IDENTAUTH = "icp4d-api/v1/authorize"
USRMGMT = "usermgmt/v1/"

def getToken(admin, passwd, url) :
    headers = {
        'Accept': 'application/json',
        'Content-type': 'application/json'
    }
    data = {
        "username" : admin,
        "password" : passwd
    }
    resp = requests.post(url + IDENTAUTH,
            data=json.dumps(data), headers=headers, verify=True)
    return(resp)

def getUsers(headersAPI, url) :
    users = "usermgmt/users?includeAll=true"

    resp = requests.get(url + USRMGMT + users, headers=headersAPI)
    return(resp)

def cre8User(headersAPI, url, data) :
    user = "user?migrated=false"
    resp = requests.post(url + USRMGMT + user, json=data, headers=headersAPI)
    return(resp)

def addUserToGroup(headersAPI, url, group_id, user_id):
    group="usermgmt/v2/groups/{}/members".format(group_id)
    uid = int(user_id)
    data = {
        "user_identifiers": [ uid ]
    }
    resp = requests.post(url + group,json=data, headers=headersAPI, verify=True)
    return(resp)

def deleteUser(headersAPI, url, username) :
    user = "user/" + username
    resp = requests.delete(url + USRMGMT + user, headers=headersAPI)
    return(resp)

## Get an access token
Note that the token usually lasts for only one hour.

An access token is used to identify a user in API requests.

In [ ]:
token = "invalid"
resp = getToken(username, password, cpd_url) # from cell-2
if resp.status_code > 202:  # if error
    print("getToken status code: {}, reason: {}".format(resp.status_code, resp.reason))
else :
    resp_json = resp.json()
    token = resp_json['token']
if token != "invalid" :
    print("Got a token at {} GMT".format(datetime.now().time().isoformat("seconds")))

# Header to use in subsequent queries
headersAPI = {
    'accept': 'application/json',
    'Content-type': 'application/json',
    'Authorization': 'Bearer ' + token,
    'cache-control': 'no-cache'
}

## List all users
This returns an array of users with a lot of information.<br/>
Only part of this information is displayed. See the documentation for details.

In [ ]:
users = []
resp = getUsers(headersAPI, cpd_url)
if resp.status_code > 202:  # if error
    print("getUsers status code: {}, reason: {}".format(resp.status_code, resp.reason))
else :
    users = resp.json()
# Assuming it worked...
myFormat = "{:18} {:10} {:24} {:28} {:8} {}"
print(myFormat.format("Display name", "uid", "username", "email", "role", "status"))
for user in users :
    print(myFormat.format(user['displayName'], user['uid'], user['username'], 
                          user['email'],user['role'], user['current_account_status']))
    #print("\tPermissions: {}".format(",".join([item for item in user['permissions'] ])))
    #print("\tUser_roles: {}".format(",".join([item for item in user['user_roles'] ])))
    print("\tGroups: {}".format(",".join([item['name'] for item in user['groups']])))

In [ ]:
# print(json.dumps([item for item in users if item['username'] == "jacques"][0], indent=4))

## Create a user

In [ ]:
user_email = "user1@company.com"

data = {
    "username": user_email,
    "authenticator": "default",
    "deletable": True,
    "displayName": user_email.split("@")[0],
    "email": user_email,
    "role": "User", # available values: ['Admin', 'User']
    "user_roles": [
        "User"
    ],
    "password": user_email.split("@")[0]
}

resp = cre8User(headersAPI, cpd_url, data)
if resp.status_code > 202:  # if error
    print("cre8User Status code: {}, reason: {}".format(resp.status_code, resp.reason))
else :
    print("Go back to listing users to see it was added")

## Delete a user

In [ ]:
resp = deleteUser(headersAPI, cpd_url, user_email)
if resp.status_code > 202:  # if error
    print("deleteUser ({}) Status code: {}, reason: {}".format(user_email, resp.status_code, resp.reason))
else :
    print("User {} deleted. Return to listing users to see it was deleted.".format(user_email))

### Author
**Jacques Roy** is a member of the IBM Enablement for Data and AI

Copyright © 2023. This notebook and its source code are released under the terms of the MIT License.